# Semantic Parser CGG in Python

## Utils.py

In [110]:
!pip install symengine

In [111]:
# Utils.py

def find_first(pred, iterable):
    for element in iterable:
        if pred(element):
            return element
    return None

def def_builder(names, constructor):
    clean_names = [name.split('.')[-1] for name in names]
    assignments = [f"{name} = {constructor(clean_name)}" for name, clean_name in zip(names, clean_names)]
    return "\n".join(assignments)


## LambdaUtils.py

In [112]:
from typing import Union, Set

# Types #

class LambdaTerm:
    pass

class Constant(LambdaTerm):
    def __init__(self, name):
        self.name = name

    def __str__(self):
        return self.name


class Variable(LambdaTerm):
    def __init__(self, name):
        self.name = name

    def __str__(self):
        return str(self.name)

Identifier = Union[Constant, Variable]

class Abstraction(LambdaTerm):
    def __init__(self, var, body):
        self.var = var
        self.body = body

    def __str__(self):
        return f"λ{self.var}.({self.body})"

class Application(LambdaTerm):
    def __init__(self, operator, operand):
        self.operator = operator
        self.operand = operand

    def __str__(self):
        return f"({self.operator})({self.operand})"

def operator(app: Application):
    return app.operator

def operand(app: Application):
    return app.operand

# Construction Sugar and IO #

def variable(*names):
    return def_builder(names, "Variable")

def constant(*names):
    return def_builder(names, "Constant")

def lambda_(var, body):
    if isinstance(var, Variable):
        return Abstraction(var, body)
    else:
        if isinstance(body, LambdaTerm):
            return lambda_(Variable(var), body)
        else:
            return lambda_(Variable(var), Variable(body))

def apply(operator, operand):

    if isinstance(operator, LambdaTerm):
        if isinstance(operand, LambdaTerm):
            return Application(operator, operand)
        else:
            return operator(Variable(operand))
    elif isinstance(operator, Predicate):
        if isinstance(operand, LambdaTerm):
            return PredicateFormula(predicate, operand)
    else:
        if isinstance(operand, LambdaTerm):
            return apply(Variable(operator),operand)
        else:
            return apply(Variable(operator),Variable(operand))


In [113]:
def rename_var(term, frm, to):
    if isinstance(term, Identifier):
        if term == frm:
            return to
        else:
            return term
    elif isinstance(term, Abstraction):
        return lambda_(rename_var(term.var, frm, to), rename_var(term.body, frm , to))
        # return Abstraction(rename_var(term.var, frm, to), substitute(term.body, frm, to))
    elif isinstance(term, PredicateFormula):
        return PredicateFormula(rename_var(term.predicate, frm, to), *[rename_var(arg, frm, to) for arg in term.arguments])
    elif isinstance(term, Application):
        return apply(rename_var(term.operator, frm, to), (rename_var(term.operand, frm, to)))
        # return Application(substitute(app.operator, frm, to), substitute(app.operand, frm, to))
    elif isinstance(term, Conjunction):
        return and_(*(rename_var(t, frm, to) for t in term.conjuncts))
    else:
        return term


def replacement(term, frm, to):

    if isinstance(term, Constant):
        return term

    elif isinstance(term, Variable):
        if term == frm:
            return to
        else:
            return term

    elif isinstance(term, Abstraction):
        if term.var == frm:
            return term
        elif term.var not in free_vars(to) and frm not in free_vars(term):
            return lambda_(term.var, replacement(term.body, frm, to))
        else:
            new_var = gen_new_var(term, to, frm)
            return lambda_(new_var, replacement(rename_var(term.body, term.var, new_var), frm, to))

    elif isinstance(term, Application):
        return apply(replacement(term.operator, frm, to), (replacement(term.operand, frm, to)))

    elif isinstance(term, PredicateFormula):
        return PredicateFormula(replacement(term.predicate, frm, to), *[replacement(arg, frm, to) for arg in term.arguments])
    elif isinstance(term, Conjunction):
        return and_(*(replacement(t, frm, to) for t in term.conjuncts))
    else:
        return term


# beta reduce operator and operand as much as possible
def chek_oper_App_or_Abs(operator, operand):

    if (isinstance(operator, Application) | isinstance(operand, Application)) == True:

          if (isinstance(operator, Application) & isinstance(operand, Application)) == True:
              return beta_reduce(operator), beta_reduce(operand)

          if isinstance(operator, Application):
              operator2 = beta_reduce(operator)
          else:
              operator2 = operator

          if isinstance(operand, Application):
              operand2 = beta_reduce(operand)
          else:
              operand2 = operand

          return operator2, operand2
    else:
        operator2 = operator
        operand2 = operand

    if ((operator2 == operator) & (operand2 == operand)) == True:
        return operator2, operand2
    if (isinstance(operator2, Application) | isinstance(operand2, Application)) == True:
        chek_oper_App_or_Abs(operator, operand)

In [114]:
class Predicate:
    def __init__(self, name):
        self.name = Constant(name)

    def __str__(self):
        return str(self.name)

def predicate(*args):
    clean_names = [str(name).split('.')[-1] for name in args[:-1]]
    arity = args[-1]
    for name, clean_name in zip(args[:-1], clean_names):
        globals()[clean_name] = Predicate(clean_name)

class PredicateFormula(LambdaTerm):
    def __init__(self, predicate, *arguments):
        self.predicate = predicate
        self.arguments = arguments

    def __str__(self):
        args_str = ' '.join(map(str, self.arguments))
        return f"{self.predicate}({args_str})"

In [115]:
if __name__ == "__main__":
    x, y = variable("x", "y")
    f, g = constant("person", "apple")
    term = apply(f, x)
    abs_term = lambda_(x, apply(f, x))
    print(term)
    print(abs_term)

(Constant('person'))(Variable('x'))
λVariable('x').((Constant('person'))(Variable('x')))


In [116]:
if __name__ == "__main__":
    x = Variable("x")
    f = Constant("person")
    term = apply(f, x)
    abs_term = lambda_(x, apply(f, x))
    print(term)
    print(abs_term)

(person)(x)
λx.((person)(x))


In [117]:
import random
import string

def gen_new_var(*terms):

    unacceptable_vars = set().union(*(free_vars(term) for term in terms)) | set().union(*(bound_vars(term) for term in terms))
    n = ''.join(random.choice(string.ascii_letters) for _ in range(50))
    while n in unacceptable_vars:
        n = ''.join(random.choice(string.ascii_letters) for _ in range(50))
    return Variable(n)

## ModalLogic.py

In [215]:
class Conjunction(LambdaTerm):
    def __init__(self, *conjuncts):
        self.conjuncts = conjuncts

    def __str__(self):
        return f"{' ∧ '.join(map(str, self.conjuncts))}"

def and_(*terms):
    flattened = []
    for term in terms:
        if isinstance(term, Conjunction):
            flattened.extend(term.conjuncts)
        else:
            flattened.append(term)
    return Conjunction(*flattened)

In [216]:
x = Variable('x')
y = Variable('y')
z = Variable('z')

conj = and_(x, y, z)

print(conj)

x ∧ y ∧ z


In [217]:
Love = Predicate("Love")
Person = Predicate("Person")
x =  Variable('x')
y = Variable('y')
z = Variable('z')

formula1 = PredicateFormula(Person, x)
print(formula1)

formula2 = PredicateFormula(Love, x, y)
print(formula2)

new_formula = rename_var(formula2, x, z)
print(new_formula)

substituted_formula = replacement(new_formula, y, z)
print(substituted_formula)

Person(x)
Love(x y)
Love(z y)
Love(z z)


In [218]:
conjunction = Conjunction(PredicateFormula(Person, Variable('x')),
                          PredicateFormula(Person, Variable('z')))
print(conjunction)

Person(x) ∧ Person(z)


In [247]:
from typing import Union, Tuple, Dict, Set

class Term:
    pass

class Constant2(Term):
    def __init__(self, name):
        self.name = name

    def __str__(self):
        return self.name


class Variable2(Term):
    def __init__(self, name):
        self.name = name

    def __str__(self):
        return str(self.name)

Binding = Dict[Variable, Constant]


class BinaryPredicate:
    def __init__(self, name):
        self.name = name

    def __str__(self):
        return str(self.name)


def relation(*names):
    return definition_macro_builder(names, BinaryPredicate)

# def variable(*names):
#     return definition_macro_builder(names, lambda name: f'Variable("{name}")')
#
# def constant(*names):
#     return definition_macro_builder(names, lambda name: f'Constant("{name}")')

class Formula:
    pass

class BinaryPredicateFormula(Formula):
    def __init__(self, predicate, arguments):
        self.predicate = predicate
        self.arguments = arguments

    def __str__(self):
        return  f"{self.predicate}({', '.join(map(str, self.arguments))})"


class HornClause:
    def __init__(self, *formulas):
        self.antecedent = Conjunction(*formulas[:-1])
        self.consequent = formulas[-1]

    def __str__(self):
        return f"{self.antecedent} ⟹ {self.consequent}"


class ExistentialQuantifier:
    def __init__(self, var, formula):
        self.var = var
        self.formula = formula

    def __str__(self):
        return f"∃{self.var}.{self.formula}"


def apply_binding(formula: Union['Constant', 'Variable', 'BinaryPredicateFormula', 'Conjunction'],
                  binding: Dict) -> Union['Constant', 'Variable', 'BinaryPredicateFormula', 'Conjunction']:
    if isinstance(formula, Constant):
        return formula
    elif isinstance(formula, Variable):
        return binding.get(formula, formula)
    elif isinstance(formula, BinaryPredicateFormula):
        return BinaryPredicateFormula(formula.predicate, tuple(apply_binding(arg, binding) for arg in formula.arguments))
    elif isinstance(formula, Conjunction):
        return Conjunction(*(apply_binding(conjunct, binding) for conjunct in formula.conjuncts))
    else:
        return formula

In [248]:
if __name__ == "__main__":

    loves = BinaryPredicate("loves")

    x = Variable("x")
    y = Variable("y")

    prop = BinaryPredicateFormula(loves, (x,y))

    print(prop)

loves(x, y)


In [249]:
constant_a = Constant('a')
constant_b = Constant('b')
variable_x = Variable('x')
variable_y = Variable('y')
predicate_p = BinaryPredicate('P')
predicate_q = BinaryPredicate('Q')
formula1 = BinaryPredicateFormula(predicate_p, (constant_a, variable_x))
formula2 = BinaryPredicateFormula(predicate_q, (variable_x, constant_b))

conjunction = Conjunction(formula1, formula2)

binding = {variable_x: constant_a, variable_y: constant_b}

conjunction_with_binding = apply_binding(conjunction, binding)

print(conjunction_with_binding)

P(a, a) ∧ Q(a, b)


In [251]:
def test_logic_modules():
    P = Predicate("P")
    Q = Predicate("Q")
    p = PredicateFormula(P, (Variable("x")))
    q = PredicateFormula(Q, (Variable("y")))
    print("Predicate Formula p:", p)
    print("Predicate Formula q:", q)

    c1 = Conjunction(p, q)
    c2 = Conjunction(q, p)
    print("Conjunction c1:", c1)
    print("Conjunction c2:", c2)

    r = BinaryPredicate("R")
    bf1 = BinaryPredicateFormula(r, (Variable("x"), Constant("c")))
    bf2 = BinaryPredicateFormula(r, (Constant("d"), Variable("y")))
    print("Binary Predicate Formula bf1:", bf1)
    print("Binary Predicate Formula bf2:", bf2)

    p1 = Variable("p1")
    p2 = Variable("p2")
    q1 = Variable("q1")
    q2 = Variable("q2")

    h = HornClause(p1, p2, q1)
    print("Horn Clause h:", h)

    e = ExistentialQuantifier(Variable("x"), c1)
    print("Existentially Quantified Statement e:", e)


if __name__ == "__main__":
    test_logic_modules()

Predicate Formula p: P(x)
Predicate Formula q: Q(y)
Conjunction c1: P(x) ∧ Q(y)
Conjunction c2: Q(y) ∧ P(x)
Binary Predicate Formula bf1: R(x, c)
Binary Predicate Formula bf2: R(d, y)
Horn Clause h: p1 ∧ p2 ⟹ q1
Existentially Quantified Statement e: ∃x.P(x) ∧ Q(y)


In [223]:
constant_a = Constant('a')
constant_b = Constant('b')
variable_x = Variable('x')
variable_y = Variable('y')
predicate_p = BinaryPredicate('P')
predicate_q = BinaryPredicate('Q')
predicate_r = BinaryPredicate('R')
formula1 = BinaryPredicateFormula(predicate_p, (constant_a, variable_x))
formula2 = BinaryPredicateFormula(predicate_q, (variable_x, constant_b))
formula3 = BinaryPredicateFormula(predicate_r, (variable_y, constant_b))

conjunction = Conjunction(formula1, formula2, formula3)

horn_clause = HornClause(conjunction, formula3)

print(horn_clause)

P(a, x) ∧ Q(x, b) ∧ R(y, b) ⟹ R(y, b)


In [224]:
class Necessity:
    def __init__(self, formula):
        self.formula = formula

    def __str__(self):
        return f"□ {self.formula}"

class Possibility:
    def __init__(self, formula):
        self.formula = formula

    def __str__(self):
        return f"◇ {self.formula}"

class ModalFormula:
    def __init__(self, operator, formula):
        self.operator = operator
        self.formula = formula

    def __str__(self):
        return f"{self.operator}{self.formula}"

In [225]:
def test_logic_modules():
    P = BinaryPredicate("P")
    p1 = Variable("p1")
    p2 = Variable("p2")
    q1 = Variable("q1")
    q2 = Variable("q2")
    p = BinaryPredicateFormula(P, (Variable("x"), Constant("a")))

    necessity_p = Necessity(p)
    possibility_p1 = Possibility(p1)
    modal_formula = ModalFormula("◇ ", p2)
    print("Necessity: ", necessity_p)
    print("Possibility: ", possibility_p1)
    print("Modal Formula: ", modal_formula)

if __name__ == "__main__":
    test_logic_modules()

Necessity:  □ P(x, a)
Possibility:  ◇ p1
Modal Formula:  ◇ p2


In [226]:
class Negation:
    def __init__(self, formula):
        self.formula = formula

    def __str__(self):
        return f"¬({self.formula})"

class Disjunction:
    def __init__(self, formula1, formula2):
        self.formula1 = formula1
        self.formula2 = formula2

    def __str__(self):
        return f"({self.formula1} ∨ {self.formula2})"

class Biconditional:
    def __init__(self, formula1, formula2):
        self.formula1 = formula1
        self.formula2 = formula2

    def __str__(self):
        return f"({self.formula1} ⇔ {self.formula2})"

class UniversalQuantifier:
    def __init__(self, variable, formula):
        self.variable = variable
        self.formula = formula

    def __str__(self):
        return f"(∀{self.variable}.{self.formula})"

In [227]:
x = Variable("x")
y = Variable("y")
formula1 = Negation(Conjunction(x, y))
formula2 = HornClause(x, Disjunction(y, Negation(x)))
print(formula1)
print(formula2)

¬(x ∧ y)
x ⟹ (y ∨ ¬(x))


## ClassifyVars.py

In [274]:
def free_vars(term):
    if isinstance(term, Variable):
        return {term}

    elif isinstance(term, Constant):
        return set()

    elif isinstance(term, Abstraction):
        return free_vars(term.body) - {term.var}

    elif isinstance(term, Application):
        return set.union(free_vars(term.operator),free_vars(term.operand))

    elif isinstance(term, Predicate):
        return set()

    elif isinstance(term, PredicateFormula):
        return set.union(free_vars(term.predicate), *[free_vars(arg) for arg in term.arguments])

    elif isinstance(term, Conjunction):
        return set.union(free_vars(term.left) , free_vars(term.right))

    elif isinstance(term, HornClause):
        return set.union(free_vars(term.head), *[free_vars(body) for body in term.body])

    elif isinstance(term, ExistentialQuantifier):
        return free_vars(term.body) - {term.var}

    elif isinstance(term, UniversalQuantifier):
        return free_vars(term.body) - {term.var}

    elif isinstance(term, Biconditional):
        return set.union(free_vars(term.left) , free_vars(term.right))

    elif isinstance(term, HornClause):
        return set.union(free_vars(term.antecedent) , free_vars(term.consequent))

    elif isinstance(term, Disjunction):
        return set.union(free_vars(term.left) , free_vars(term.right))

    elif isinstance(term, Negation):
        return free_vars(term.term)

    elif isinstance(term, Necessity):
        return free_vars(term.term)

    elif isinstance(term, Possibility):
        return free_vars(term.term)

    else:
        return term


def bound_vars(term):
    if isinstance(term, Identifier):
        return set()

    elif isinstance(term, Abstraction):
        return set.union({term.var} , bound_vars(term.body))

    elif isinstance(term, Application):
        return set.union(bound_vars(term.operator) , bound_vars(term.operand))

    elif isinstance(term, Predicate):
        return set()

    elif isinstance(term, PredicateFormula):
        return set.union(bound_vars(term.predicate), *[bound_vars(arg) for arg in term.arguments])

    elif isinstance(term, Conjunction):
        return set.union(bound_vars(term.left) , bound_vars(term.right))

    elif isinstance(term, HornClause):
        return set.union(bound_vars(term.head), *[bound_vars(body) for body in term.body])

    elif isinstance(term, ExistentialQuantifier):
        return set.union({term.var} , bound_vars(term.body))

    elif isinstance(term, UniversalQuantifier):
        return set.union({term.var} , bound_vars(term.body))

    elif isinstance(term, Biconditional):
        return set.union(bound_vars(term.left) , bound_vars(term.right))

    elif isinstance(term, HornClause):
        return set.union(bound_vars(term.antecedent) , bound_vars(term.consequent))

    elif isinstance(term, Disjunction):
        return set.union(bound_vars(term.left) , bound_vars(term.right))

    elif isinstance(term, Negation):
        return bound_vars(term.term)

    elif isinstance(term, Necessity):
        return bound_vars(term.term)

    elif isinstance(term, Possibility):
        return bound_vars(term.term)

    else:
        return term


def all_vars(term):
    return set.union(free_vars(term) , bound_vars(term))

## LambdaCalculus.py

### α-conversion

In [132]:
# def alpha_convert(abs, new_var):
#     if isinstance(new_var, Variable):
#         if (   (new_var not in free_vars(abs)) and (new_var not in bound_vars(abs))   ):
#             return rename_var(abs, abs.var, new_var)
#         else:
#             raise ValueError(f"Invalid α-conversion: {new_var} is free or bound in {abs}")
#
#     else:
#       alpha_convert(abs, Variable(new_var))

def alpha_convert(abs, new_var):
    if isinstance(new_var, Variable):
        if new_var not in free_vars(abs) and new_var not in bound_vars(abs):
            return rename_var(abs, abs.var, new_var)
        else:
            raise ValueError(f"Invalid α-conversion: {new_var} is free or bound in {abs}")
    else:
        return alpha_convert(abs, Variable(new_var))

In [133]:
if __name__ == "__main__":
    x = Variable("x")
    y = Variable("y")
    z = Variable("z")
    abs_term = Abstraction(x, apply(x, y))

    print("Original lambda expression:", abs_term)

    new_abs = alpha_convert(abs_term, z)
    print("\nAlpha converted lambda expression:", new_abs)

    renamed_abs = rename_var(abs_term, x, z)
    print("\nLambda expression after variable renaming:", renamed_abs)

    app = Application(x, y)
    renamed_app = rename_var(app, x, z)
    print("\nApplication after variable renaming:", renamed_app)


Original lambda expression: λx.((x)(y))

Alpha converted lambda expression: λz.((z)(y))

Lambda expression after variable renaming: λz.((z)(y))

Application after variable renaming: (z)(y)


In [134]:
if __name__ == "__main__":
    x = Variable("x")
    y = Variable("y")

    abs_term = Abstraction(x, Abstraction(y, apply(x, y)))

    print("Original lambda expression:", abs_term)

    new_var = Variable("z")
    alpha_converted_abs = alpha_convert(abs_term, new_var)

    print("\nAlpha-converted lambda expression:", alpha_converted_abs)

Original lambda expression: λx.(λy.((x)(y)))

Alpha-converted lambda expression: λz.(λy.((z)(y)))


### α-equivalence

In [135]:
def alpha_equivalent(term1, term2):
      if isinstance(term1, Identifier) and isinstance(term2, Identifier):
          return term1 == term2

      elif isinstance(term1, Abstraction) and isinstance(term2, Abstraction):
          new_arg = gen_new_var(term1, term2)
          return alpha_equivalent((alpha_convert(term1, new_arg)).body, (alpha_convert(term2, new_arg)).body)

      elif isinstance(term1, Application) and isinstance(term2, Application):
          return alpha_equivalent(term1.operator, term2.operator) & alpha_equivalent(term1.operand, term2.operand)

      elif isinstance(term1, Variable) and isinstance(term2, Variable):
          stack = [(term1, term2)]
          bindings1 = {}
          bindings2 = {}
          if term1 in bindings1 and term2 in bindings2:
              if bindings1[term1] != bindings2[term2]:
                  return True
          else:
              if term1.name != term2.name:
                  return False

      elif isinstance(term1, Predicate) and isinstance(term2, Predicate):
          return term1 == term2

      elif isinstance(term1, PredicateFormula) and isinstance(term2, PredicateFormula):
          if len(term1.arguments) == len(term2.arguments) and term1.predicate== term2.predicate:
              return all(alpha_equivalent(parg, qarg) for parg, qarg in zip(term1.arguments, term2.arguments))

      elif isinstance(term1, Conjunction) and isinstance(term2, Conjunction):
          if len(term1.conjuncts) != len(term2.conjuncts):
              return False
          else:
              return all(any(alpha_equivalent(s, t) for t in term2.conjuncts) for s in term1.conjuncts) \
                    and all(any(alpha_equivalent(s, t) for t in term1.conjuncts) for s in term2.conjuncts)
      else:
            return False

In [136]:
if __name__ == "__main__":
    x = Variable("x")
    y = Variable("y")
    z = Variable("z")

    abs_term1 = Abstraction(z, apply(z, y))
    abs_term2 = Abstraction(x, apply(x, y))
    abs_term3 = Abstraction(x, apply(z, y))

    print("Original lambda expressions:")
    print("1.", abs_term1)
    print("2.", abs_term2)
    print("3.", abs_term3)

    print("\nChecking alpha equivalence of 1 and 2:")
    print("Alpha equivalent:", alpha_equivalent(abs_term1, abs_term2))

    print("\nChecking alpha equivalence of 2 and 3:")
    print("Alpha equivalent:", alpha_equivalent(abs_term2, abs_term3))

Original lambda expressions:
1. λz.((z)(y))
2. λx.((x)(y))
3. λx.((z)(y))

Checking alpha equivalence of 1 and 2:
Alpha equivalent: True

Checking alpha equivalence of 2 and 3:
Alpha equivalent: False


### β-reduction

In [137]:
def beta_reduce(term):
    if isinstance(term, Application) and isinstance(term.operator, Abstraction):
        return replacement(term.operator.body, term.operator.var, term.operand)
    else:
        return term

In [138]:
if __name__ == "__main__":
    x = Variable("x")
    y = Variable("y")
    f = Constant("f")

    abs_term = apply(Abstraction(x, apply(f, x)), y)

    print("Original lambda expression:", abs_term)

    reduced_abs = beta_reduce(abs_term)

    print("\nReduced lambda expression:", reduced_abs)

Original lambda expression: (λx.((f)(x)))(y)

Reduced lambda expression: (f)(y)


### η-reduction

In [139]:
def is_eta(abs):
    return isinstance(abs.body, Application) and abs.body.operand == abs.var

def eta_reduce(abs):
    if is_eta(abs):
        return abs.body.operator
    else:
        return abs

In [140]:
if __name__ == "__main__":
    x = Variable("x")
    y = Variable("y")
    f = Constant("f")

    abs_term1 = Abstraction(x, Application(f, x))
    abs_term2 = Abstraction(x, Application(x, y))
    abs_term3 = Abstraction(x, Application(y, x))

    reduced_abs1 = eta_reduce(abs_term1)
    reduced_abs2 = eta_reduce(abs_term2)
    reduced_abs3 = eta_reduce(abs_term3)

    print(f"First lambda expression is: {abs_term1}\n" + (f"The expression is an eta redex.\nEta Reduced lambda expression is {reduced_abs1}\n" if is_eta(abs_term1) else "The expression is not an eta redex.\n"))
    print(f"Second lambda expression is: {abs_term2}\n" + (f"The expression is an eta redex.\nEta Reduced lambda expression is {reduced_abs2}\n" if is_eta(abs_term2) else "The expression is not an eta redex.\n"))
    print(f"Third lambda expression is: {abs_term3}\n" + (f"The expression is an eta redex.\nEta Reduced lambda expression is {reduced_abs3}\n" if is_eta(abs_term3) else "The expression is not an eta redex.\n"))

First lambda expression is: λx.((f)(x))
The expression is an eta redex.
Eta Reduced lambda expression is f

Second lambda expression is: λx.((x)(y))
The expression is not an eta redex.

Third lambda expression is: λx.((y)(x))
The expression is an eta redex.
Eta Reduced lambda expression is y



### δ-reduction

In [141]:
def delta_reduce(term):
    if isinstance(term, Application) and isinstance(term.operator, Constant):
        constant = term.operator
        if constant.name == 'add':
            return Constant('(+ ' + str(term.operand.operator) + ' ' + str(term.operand.operand) + ')')
        elif constant.name == 'multiply':
            return Constant('(* ' + str(term.operand.operator) + ' ' + str(term.operand.operand) + ')')
        else:
            return term
    else:
        return term

In [142]:
add = Constant('add')
multiply = Constant('multiply')

x = Variable('x')
y = Variable('y')

add_app = Application(add, Application(x, y))
multiply_app = Application(multiply, Application(x, y))

print("Original expression:", add_app)
print("Delta reduced summation:", delta_reduce(add_app), "\n")

print("Original expression:", multiply_app)
print("Delta reduced multiplication:", delta_reduce(multiply_app))

Original expression: (add)((x)(y))
Delta reduced summation: (+ x y) 

Original expression: (multiply)((x)(y))
Delta reduced multiplication: (* x y)


### Normal Order

In [143]:
def normal_order(term):
    if isinstance(term, Application):
        operator = normal_order(term.operator)
        if isinstance(operator, Abstraction):
            operand = normal_order(term.operand)
            return normal_order(replacement(operator.body, operator.var, operand))
        else:
            return Application(operator, normal_order(term.operand))
    elif isinstance(term, Abstraction):
        return Abstraction(term.var, normal_order(term.body))
    else:
        return term

In [144]:
x = Variable('x')
y = Variable('y')
f = Variable('f')

abs1 = Abstraction(x, Application(f, x))
abs2 = Abstraction(y, Application(y, y))
app = Application(abs1, abs2)

print("Original term:", app)

reduced_term = normal_order(app)

print("Reduced term using normal order reduction:")
print(reduced_term)

reduced_term2 = beta_reduce(app)
print("Reduced term using beta reduction:")
print(reduced_term2)

Original term: (λx.((f)(x)))(λy.((y)(y)))
Reduced term using normal order reduction:
(f)(λy.((y)(y)))
Reduced term using beta reduction:
(f)(λy.((y)(y)))


In [145]:
x = Variable("x")
y = Variable("y")
z = Variable("z")

lambda_expr = Application(
    Abstraction(x, Abstraction(y, Application(x, y))),
    Abstraction(z, Application(z, z))
)

normal_order_reduced = normal_order(lambda_expr)
print("Reduced term using normal order reduction:")
print(normal_order_reduced)

beta_reduced = beta_reduce(lambda_expr)
print("\nReduced term using beta reduction:")
print(beta_reduced)

Reduced term using normal order reduction:
λKSlINMhoOqXkgqIjfAtQALHcMqPnKdIONXcAImHEdWNGwLeXmg.((KSlINMhoOqXkgqIjfAtQALHcMqPnKdIONXcAImHEdWNGwLeXmg)(KSlINMhoOqXkgqIjfAtQALHcMqPnKdIONXcAImHEdWNGwLeXmg))

Reduced term using beta reduction:
λCLzQkDqrUDIahOAcXzHlplyGgNarezZfJRGFpetQaFCOKZLRez.((λz.((z)(z)))(CLzQkDqrUDIahOAcXzHlplyGgNarezZfJRGFpetQaFCOKZLRez))


### Misc.

In [146]:
def lambda_reduce(term, prev_term=None):
    if prev_term is not None and term == prev_term:
        return term

    if isinstance(term, Application):
        new_operator = term.operator
        new_operand = term.operand
        new_operator, new_operand = chek_oper_App_or_Abs(new_operator, new_operand)

        return beta_reduce(apply(new_operator, new_operand))


    elif isinstance(term, Abstraction):
        if alpha_equivalent(term, prev_term):
            return term
        else:
            simplified = eta_reduce(term)
            if isinstance(simplified, Abstraction):
                return lambda_reduce(lambda_(simplified.var, lambda_reduce(simplified.body)), term)
                # return lambda_reduce(Abstraction(simplified.var, lambda_reduce(simplified.body)), term)
            else:
                return lambda_reduce(simplified)

            # simplified_args = [lambda_reduce(arg) for arg in term.arguments]
            # return lambda_reduce(apply(term.predicate, *simplified_args), term)

    elif isinstance(term, PredicateFormula):
        if alpha_equivalent(term, prev_term):
            term
        else:
            simplified_args = [lambda_reduce(arg) for arg in term.arguments]
            return lambda_reduce(term.predicate(*simplified_args), term)

    elif isinstance(term, Conjunction):
        simplified_conjs = [lambda_reduce(t) for t in term.conjuncts]
        return Conjunction(*simplified_conjs)

    else:
        return term

def compose(f, g):
    new_var = gen_new_var(f, g)
    return Abstraction(new_var, lambda_reduce(apply(f(g(new_var)))))

def final_rename(term):
    term_vars = all_vars(term)
    nice_vars = set(Variable('a' + str(i)) for i in range(26)) - term_vars
    for v in filter(lambda v: str(v).startswith('#'), term_vars):
        term = rename_var(term, v, nice_vars.pop())
    return term

In [147]:
if __name__ == "__main__":
    x = Variable("x")
    y = Variable("y")
    f = Variable("f")
    abs1 = Abstraction(x, Application(f, x))
    abs2 = apply(Abstraction(x, apply(f, x)), y)
    abs3 = apply(apply(Abstraction(x, apply(f, x)), y), apply(Abstraction(x, apply(f, x)), y))

    print("Original lambda expression:", abs1)
    reduced_abs1 = lambda_reduce(abs1)
    print("Reduced lambda expression:", reduced_abs1)

    print("\nOriginal lambda expression:", (abs2))
    reduced_abs2 = lambda_reduce(abs2)
    print("Reduced lambda expression:", reduced_abs2)

    print("\nOriginal lambda expression:", abs3)
    reduced_abs3 = lambda_reduce(abs3)
    print("Reduced lambda expression:", reduced_abs3)


Original lambda expression: λx.((f)(x))
Reduced lambda expression: f

Original lambda expression: (λx.((f)(x)))(y)
Reduced lambda expression: (f)(y)

Original lambda expression: ((λx.((f)(x)))(y))((λx.((f)(x)))(y))
Reduced lambda expression: ((f)(y))((f)(y))


## CCGs

In [252]:
from typing import Union, List

class Direction:
    pass

class Right(Direction):
    pass

class Left(Direction):
    pass

class SyntacticType:
    def __str__(self):
        return self.__class__.__name__

    def __truediv__(self, other: 'SyntacticType'):
        return ComplexSyntacticType(self, other, Right())

    def __floordiv__(self, other: 'SyntacticType'):
        return ComplexSyntacticType(other, self, Left())

    def agrees_with(self, other):
        return False

class ComplexSyntacticType(SyntacticType):
    def __init__(self, res: SyntacticType, arg: SyntacticType, direction: Direction):
        self.res = res
        self.arg = arg
        self.direction = direction

    def __str__(self):
        direction = '/' if isinstance(self.direction, Right) else '\\'
        return f"{self.res} {direction} {self.arg}"

    def agrees_with(self, other):
        if isinstance(other, ComplexSyntacticType):
            return (self.direction == other.direction and
                    self.res.agrees_with(other.res) and
                    self.arg.agrees_with(other.arg))
        elif isinstance(other, PrimitiveSyntacticType):
            return False

In [253]:
np = SyntacticType()
vp = SyntacticType()
s = SyntacticType()

np_vp_s_right = np // (vp / s)
np_vp_s_left = np // vp / s

print("Right Operator:", np_vp_s_right)
print("Left Operator:", np_vp_s_left)

Right Operator: SyntacticType / SyntacticType \ SyntacticType
Left Operator: SyntacticType \ SyntacticType / SyntacticType


In [254]:
class PrimitiveSyntacticType(SyntacticType):
    def __init__(self, name: str, features: List[str] = None):
        self.name = name
        self.features = features or []

    def agrees_with(self, other):
        if isinstance(other, PrimitiveSyntacticType):
            return self.name == other.name and all(feature in other.features for feature in self.features)
        return False

    def __call__(self, *features_: str):
        return PrimitiveSyntacticType(self.name, self.features + list(features_))

    def __str__(self):
        if self.features:
            return f"{self.name}({','.join(self.features)})"
        return self.name

In [255]:
np = PrimitiveSyntacticType("NP")
n_sg = PrimitiveSyntacticType("N", ["sg"])
v = PrimitiveSyntacticType("V")
v_pl = v("pl")

np_vp_s_right = np / v // n_sg
np_vp_s_left = np // v / n_sg

print("Right Operator:", np_vp_s_right)
print("Left Operator:", np_vp_s_left)

print("Agreement between Noun and Verb:", n_sg.agrees_with(v))
print("Agreement between Noun and Plural Verb:", n_sg.agrees_with(v_pl))
print("Agreement between Complex Types:", np_vp_s_right.agrees_with(np_vp_s_left))


Right Operator: N(sg) \ NP / V
Left Operator: V \ NP / N(sg)
Agreement between Noun and Verb: False
Agreement between Noun and Plural Verb: False
Agreement between Complex Types: False


In [256]:
class Category:
    def __init__(self, syntactic_type:SyntacticType, logical_form:LambdaTerm):
        self.syntactic_type = syntactic_type
        self.logical_form = logical_form

    def __repr__(self):
        return f"Category({self.syntactic_type}, {self.logical_form})"

def agrees(p, q):
    if isinstance(p, PrimitiveSyntacticType) and isinstance(q, PrimitiveSyntacticType):
        return p.name == q.name and all(feature in q.features for feature in p.features)
    elif isinstance(p, PrimitiveSyntacticType) and isinstance(q, ComplexSyntacticType):
        return False
    elif isinstance(p, ComplexSyntacticType) and isinstance(q, PrimitiveSyntacticType):
        return False
    elif isinstance(p, ComplexSyntacticType) and isinstance(q, ComplexSyntacticType):
        return (p.direction == q.direction and
                p.res.agrees_with(q.res) and
                p.arg.agrees_with(q.arg))
    elif isinstance(c1, Category) and isinstance(c2, Category):
        return (agrees(c1.syntactic_type, c2.syntactic_type) and
                lambda_reduce(c1.logical_form) == lambda_reduce(c2.logical_form))
    elif isinstance(c1, Category) and isinstance(c2, ComplexSyntacticType):
        return False
    elif isinstance(c1, ComplexSyntacticType) and isinstance(c2, Category):
        return False
    else:
        return False

In [257]:
def primitive_syntactic_type(*names):
    def constructor(name):
        return PrimitiveSyntacticType(name, [])

    def def_builder2(names, constructor):
        clean_names = [name.split(".")[-1] for name in names]
        return [
            constructor(clean_name)
            for name, clean_name in zip(names, clean_names)
        ]

    return def_builder2(names, constructor)

In [258]:
if __name__ == "__main__":
    types = primitive_syntactic_type("NP", "VP", "N", "V")
    for t in types:
        print(t)

NP
VP
N
V


In [259]:
p1 = PrimitiveSyntacticType("NP")
p2 = PrimitiveSyntacticType("N", ["sg"])
p3 = PrimitiveSyntacticType("V")

c1 = ComplexSyntacticType(p1, p2, Right())
c2 = ComplexSyntacticType(p2, p3, Left())

x = Variable("x")
y = Variable("y")
z = Constant("z")

abs_term1 = Abstraction(x, Abstraction(y, apply(x, y)))
abs_term2 = alpha_convert(abs_term1, z)

cat1 = Category(p1, abs_term1)
cat2 = Category(p2, abs_term2)

print(agrees(p1, p2))
print(agrees(c1, c2))
print(cat1)
print(cat2)
print(agrees(cat1, cat2))

False
False
Category(NP, λx.(λy.((x)(y))))
Category(N(sg), λz.(λy.((z)(y))))
False


In [260]:
class LexicalEntry:
    def __init__(self, lexeme, cat):
        self.lexeme = lexeme
        self.cat = cat

def LexicalEntry(lexeme, syn_type, lf):
    return LexicalEntry(lexeme, Category(syn_type, lf))

def lexeme(lexical_entry):
    return lexical_entry.lexeme

def category(lexical_entry):
    return lexical_entry.cat

class Lexicon:
    def __init__(self, lexical_entries):
        self.lexical_entries = lexical_entries

def lexical_entries(lexicon):
    return lexicon.lexical_entries

def merge(*lexicons):
    merged_entries = set()
    for lexicon in lexicons:
        merged_entries.update(lexical_entries(lexicon))
    return Lexicon(list(merged_entries))

class CombinatoryCategorialGrammar:
    def __init__(self, lexicon, type_shifting_rules):
        self.lexicon = lexicon
        self.type_shifting_rules = type_shifting_rules

def lexicon(g):
    return g.lexicon

def type_shifting_rules(g):
    return g.type_shifting_rules

def type_shifted_lexical_entries(g):
    result = list(lexical_entries(lexicon(g)))
    for entry in lexical_entries(lexicon(g)):
        new_categories = type_shifting_rules(g).get(lexeme(entry), [])
        for new_category in new_categories:
            result.append(LexicalEntry(lexeme(entry), new_category))
    return result

In [261]:
class LexicalEntry:
    def __init__(self, lexeme, cat):
        self.lexeme = lexeme
        self.cat = cat

def LexicalEntry(lexeme, syn_type, lf):
    return LexicalEntry(lexeme, Category(syn_type, lf))

def lexeme(lexical_entry):
    return lexical_entry.lexeme

def category(lexical_entry):
    return lexical_entry.cat

class Lexicon:
    def __init__(self, lexical_entries):
        self.lexical_entries = lexical_entries

    def lexical_entries(self):
        return self.lexical_entries

    def iterate(self, state=None):
        return iter(self.lexical_entries)

    def merge(*lexicons):
        merged_entries = set()
        for lexicon in lexicons:
            merged_entries.update(set(lexicon.lexical_entries))
        return Lexicon(list(merged_entries))

class CombinatoryCategorialGrammar:
    def __init__(self, lexicon, type_shifting_rules):
        self.lexicon = lexicon
        self.type_shifting_rules = type_shifting_rules

    def lexicon(self):
        return self.lexicon

    def type_shifting_rules(self):
        return self.type_shifting_rules

    def type_shifted_lexical_entries(self):
        result = []
        for entry in self.lexicon.lexical_entries:
            for new_category in self.type_shifting_rules.get(entry.lexeme, []):
                result.append(LexicalEntry(entry.lexeme, new_category))
        return result

In [262]:
class LexicalEntry:
    def __init__(self, lexeme, cat):
        self.lexeme = lexeme
        self.cat = cat

    def __str__(self):
        return f"LexicalEntry: {self.lexeme}: {self.cat}"

    @classmethod
    def from_syn_type(cls, lexeme, syn_type, lf):
        return cls(lexeme, Category(syn_type, lf))


def lexeme(lexical_entry):
    return lexical_entry.lexeme

def category(lexical_entry):
    return lexical_entry.cat

class Lexicon:
    def __init__(self, lexical_entries):
        self.lexical_entries = lexical_entries

    def __str__(self):
        entries_str = '\n'.join(str(entry) for entry in self.lexical_entries)
        return f"Lexicon entries:\n{entries_str}"

def merge(*lexicons):
    merged_entries = set()
    for lexicon in lexicons:
        merged_entries.update(lexical_entries(lexicon))
    return Lexicon(list(merged_entries))

class CombinatoryCategorialGrammar:
    def __init__(self, lexicon, type_shifting_rules):
        self.lexicon = lexicon
        self.type_shifting_rules = type_shifting_rules

def lexicon(g):
    return g.lexicon

def type_shifting_rules(g):
    return g.type_shifting_rules

def type_shifted_lexical_entries(g):
    lex = lexicon(g)
    result = lex.lexical_entries  # Remove the method call
    for entry in lex.lexical_entries:    # Call as an attribute
        new_categories = g.type_shifting_rules.get(lexeme(entry), [])
        # Process new categories as needed
    return result



In [263]:
x = Variable("x")
y = Variable("y")
f = Constant("person")
g = Constant("person")
doctor = Constant("doctor")
prist = Constant("prist")
term = apply(f, x)
abs_term = lambda_(x, apply(f, x))

N = PrimitiveSyntacticType("N", [])
V = PrimitiveSyntacticType("V", [])
NP = PrimitiveSyntacticType("NP", [])
VP = PrimitiveSyntacticType("VP", [])

lf1 = lambda_(x, apply(f, x))
lf2 = lambda_(x, apply(g, x))
lf3 = lambda_(x, HornClause(apply(doctor, x), apply(g, x)))
lf4 = lambda_(x, Conjunction(apply(prist, x), apply(f, x)))

entry1 = LexicalEntry.from_syn_type("dog", N, lf1)
entry2 = LexicalEntry.from_syn_type("run", V, lf2)
entry3 = LexicalEntry.from_syn_type("the", NP // N, lf3)
entry4 = LexicalEntry.from_syn_type("quickly", (NP / VP) // NP, lf4)

lexical_entries = [entry1, entry2, entry3, entry4]
lexicon_instance = Lexicon(lexical_entries)

type_shifting_rules_instance = {
    "quickly": [Category((N / NP) // (N // NP), lf4)],
    "run": [Category(VP // VP, lf2)]
}

In [264]:
ccg = CombinatoryCategorialGrammar(lexicon_instance, type_shifting_rules_instance)

type_shifted_entries = type_shifted_lexical_entries(ccg)

In [265]:
print(dir(ccg))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'lexicon', 'type_shifting_rules']


In [266]:
print(ccg.type_shifting_rules)

{'quickly': [Category(NP \ N \ N / NP, λx.((prist)(x) ∧ (person)(x)))], 'run': [Category(VP \ VP, λx.((person)(x)))]}


In [267]:
print(ccg.lexicon)

Lexicon entries:
LexicalEntry: dog: Category(N, λx.((person)(x)))
LexicalEntry: run: Category(V, λx.((person)(x)))
LexicalEntry: the: Category(N \ NP, λx.((doctor)(x) ⟹ (person)(x)))
LexicalEntry: quickly: Category(NP \ NP / VP, λx.((prist)(x) ∧ (person)(x)))


In [268]:
def simplify(term):
    term_vars = all_vars(term)
    simplified_vars = set(map(lambda i: Variable('a' + str(i)), range(26))) - term_vars
    simplified = reduce(lambda term, v: rename_var(term, v, simplified_vars.pop()),
                             filter(lambda v: str(v).startswith('#'), term_vars),
                             init=term)
    return simplified



##########  This part is not functional yet ############


def Apply(left, right, direction):
    operator, operand = (left, right) if isinstance(direction, Right) else (right, left)
    syntactic_type_combinator = Right if isinstance(direction, Right) else Left

#    if isinstance(operator.syntactic_type, ComplexSyntacticType) and \
#       agrees(operator.syntactic_type.arg, operand.syntactic_type):
    return Category((operator.syntactic_type.res), simplify(lambda_reduce(apply(operator.logical_form, operand.logical_form))))


def Compose(left, right, direction):
    operator, operand = (left, right) if isinstance(direction, Right) else (right, left)
    syntactic_type_combinator = "/" if isinstance(direction, Right) else "\\"

#    if isinstance(operator.syntactic_type, ComplexSyntacticType) and \
#       isinstance(operand.syntactic_type, ComplexSyntacticType) and \
#       agrees(operator.syntactic_type.arg, operand.syntactic_type.res):
    return Category(syntactic_type_combinator(operator.syntactic_type.res, operand.syntactic_type.arg),
                        simplify(Compose(operator.logical_form, operand.logical_form)))

In [ ]:
##########  This part is not functional yet ############

x = Variable("x")
y =  Variable("y")
f = Constant("f")
g = Constant("g")

NP = ComplexSyntacticType(N, NP, Right)
VP = ComplexSyntacticType(V, NP, Left)

N = PrimitiveSyntacticType("N")
V = PrimitiveSyntacticType("V")

lf1 = LambdaTerm()
lf2 = LambdaTerm()

cat1 = Category(N, lf1)
cat2 = Category(NP, lf2)

result = Apply(cat1, cat2, Right)

print("Result of applying cat1 to cat2 (Right):", result)


## SemanticParser


## Semantic Parser Demo